# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import datetime
# from selenium.webdrive.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True
# navegador = webdriver.Chrome(options=chrome_options)

navegador = webdriver.Chrome()

# Passo 1: Pegar a cotação do dolar
# - entrar no google
navegador.get("https://www.google.com/")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print("dolar", "=", cotacao_dolar)

# Passo 2: Pegar a cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print("euro", "=", cotacao_euro)

# Passo 3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")
navegador.find_element('xpath', '//*[@id="comercial"]')

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print("ouro", "=", cotacao_ouro)

navegador.quit()

dolar = 5.2172
euro = 5.553892002
ouro = 314.11


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

- Atualizando os preços e o cálculo do Preço Final

In [4]:
# Passo 4: Importar a base de dados e Atualizar a base
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

print(tabela)


         Produtos  Preço Original  Moeda  Cotação  Preço de Compra  Margem  \
0    Câmera Canon          999.99  Dólar        5          4999.95    1.40   
1   Carro Renault         4500.00   Euro        6         27000.00    2.00   
2   Notebook Dell          899.99  Dólar        5          4499.95    1.70   
3          IPhone          799.00  Dólar        5          3995.00    1.70   
4      Carro Fiat         3000.00   Euro        6         18000.00    1.90   
5  Celular Xiaomi          480.48  Dólar        5          2402.40    2.00   
6        Joia 20g           20.00   Ouro      350          7000.00    1.15   

   Preço de Venda  
0        6999.930  
1       54000.000  
2        7649.915  
3        6791.500  
4       34200.000  
5        4804.800  
6        8050.000  


In [5]:
# Passo 5: Recalcular os preços

# atualizar cotação
# - cotação dolar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
# - cotação euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
# - cotação ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# recalcular os preços
# preço de compra
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# preço de venda
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.217200,5217.147828,1.40,7304.006959
1,Carro Renault,4500.00,Euro,5.553892,24992.514009,2.00,49985.028018
2,Notebook Dell,899.99,Dólar,5.217200,4695.427828,1.70,7982.227308
3,IPhone,799.00,Dólar,5.217200,4168.542800,1.70,7086.522760
4,Carro Fiat,3000.00,Euro,5.553892,16661.676006,1.90,31657.184411
5,Celular Xiaomi,480.48,Dólar,5.217200,2506.760256,2.00,5013.520512
6,Joia 20g,20.00,Ouro,314.110000,6282.200000,1.15,7224.530000


### Agora vamos exportar a nova base de preços atualizada

In [4]:
# Passo 6: Exportar a base atualizada


tabela.to_excel("Produtos_NOVO.xlsx", index=False)

In [5]:
!pip install selenium